In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize
import datetime

In [2]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [68]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["jour (0)"]=df.apply(lambda row: monthCorrect(row['jour (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)

In [69]:
max_quinzaine=24
max_month=12
max_area_lat=12
max_area_lon=31
number_grid=(max_area_lat+1)*(max_area_lon+1)
print df.head(2)

   Annee (0) jour (0) mois (0)  dist cote (nq) zone lat (0)  long(0)  \
0       2010       28       10             NaN  BCD  12:08N  054:25E   
1       2010       30       10             NaN  BCD  15:06N  055:58E   

        type(0)    pavillon statut de l'attaque(0)   ...    quinzaine_week  \
0          Dhow  Not Stated               Steaming   ...                20   
1  Bulk Carrier       Malta               Steaming   ...                20   

                                       newDescriptif  Guns  mousson  \
0                           pirates hijacked a dhow.     0        0   
1  armed pirates attacked and boarded the ship un...     0        0   

   distance au port  distance_shorline  lat_area  lon_area  large_area  \
0        534.679104          35.872306         4        31           G   
1        292.330113         233.517911        10        34           G   

      date  
0  2010-10  
1  2010-10  

[2 rows x 33 columns]


In [19]:
df['lat_area'] = df['lat_area'].astype(int)
df['lon_area'] = df['lon_area'].astype(int)

# Version 1 : Création d'une matrice en regroupant par quinzaine et zone

In [5]:
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [52]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week","area"] ).size()}).reset_index()

In [53]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head()

   Annee (0)  quinzaine_week  area  count lat_area lon_area
0       2008               3  4_20      2        4       20
1       2008               6  3_21      1        3       21
2       2008               6  4_18      1        4       18
3       2008               6  6_19      1        6       19
4       2008               7  5_16      1        5       16


In [54]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
Colyears=np.repeat(uniqueYears, 24*(max_area_lon+1)*(max_area_lat+1))

TwoweekRepeat=np.repeat(np.linspace(1, max_quinzaine, num=max_quinzaine), (max_area_lon+1)*(max_area_lat+1))
ColTwoweek=np.tile(TwoweekRepeat, len(uniqueYears))

latRepeat=np.linspace(0, max_area_lat,num=max_area_lat+1)
ColLatArea=np.repeat(latRepeat, max_area_lon+1)
ColLatArea=np.tile(ColLatArea, max_quinzaine)
ColLatArea=np.tile(ColLatArea, len(uniqueYears))

#print ColLatArea

lonRepeat=np.tile(np.linspace(0, max_area_lon, num=max_area_lon+1),max_area_lat+1)
ColLonArea=np.tile(lonRepeat, max_quinzaine)
ColLonArea=np.tile(ColLonArea, len(uniqueYears))

print ColLonArea

count=np.repeat(0, len(Colyears))

print len(Colyears)
print len(ColTwoweek)
print len(ColLatArea)
print len(ColLonArea)


[  0.   1.   2. ...,  29.  30.  31.]
69888
69888
69888
69888


## Fill the matrice

In [55]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     '2_week': ColTwoweek, 
     'lat_area': ColLatArea,
     'lon_area': ColLonArea,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal['lat_area']=dfFinal['lat_area'].astype(int).astype('str')
dfFinal['lon_area']=dfFinal['lon_area'].astype(int).astype('str')

print dfFinal.head()

   2_week  Years  count lat_area lon_area
0       1   2008      0        0        0
1       1   2008      0        0        1
2       1   2008      0        0        2
3       1   2008      0        0        3
4       1   2008      0        0        4


In [56]:
for index, row in dfAgregateTotal.iterrows():
    if index%100==0:
        print index
    #print row['Annee (0)'], row['quinzaine_week'], row['count'] ,  row['lat_area'] , row['lon_area'] 
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) & (dfFinal['lat_area']==row['lat_area']) & (dfFinal['lon_area']==row['lon_area'] )& (dfFinal['2_week']==row['quinzaine_week'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal.set_value(idx.index.values, 'count',row['count'])

0
100
200
300


In [38]:
dfFinal['count'].sum()

421

## Write matrice in a file

In [57]:
dfFinal.to_csv("../data/aden_golfe_matrix_area.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 2 : Création d'une matrice en regroupant par mois et par type

In [106]:
dfAgregateTotal2=pd.DataFrame({'count' : df.groupby( [ "date","type d'attaque(0)"] ).size()}).reset_index()
print dfAgregateTotal2.head(10)

      date type d'attaque(0)  count
0  2008-02        Fired upon      1
1  2008-02          Hijacked      1
2  2008-03         Attempted      1
3  2008-03           Boarded      1
4  2008-03        Fired upon      1
5  2008-04         Attempted      1
6  2008-04        Fired upon      3
7  2008-04          Hijacked      2
8  2008-05         Attempted      2
9  2008-05        Fired upon      2


## Matrix empty

In [107]:
col_years=np.repeat(np.sort(df["Annee (0)"].unique()), max_month)

month_repeat=np.linspace(1, max_month, num=max_month)
month=np.tile(month_repeat, len(unique_years))

col_count=np.repeat(0, len(month))


dic2 = {'Years': col_years, 
     'Month': month, 
     'Fired upon': col_count,
     'Hijacked': col_count,
     'Attempted': col_count,
     'Boarded': col_count
       }

dfFinal_2 = pd.DataFrame(dic2)
dfFinal_2.head()
dfFinal_2["Month"]=dfFinal_2.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfFinal_2["date"]=dfFinal_2.apply(lambda row: str(row["Years"]) +"-"+ str(row["Month"]), axis=1)
dfFinal_2 = dfFinal_2.drop('Month', 1)
dfFinal_2 = dfFinal_2.drop('Years', 1)
dfFinal_2.head()

,Attempted,Boarded,Fired upon,Hijacked,date
0,0,0,0,0,2008-01
1,0,0,0,0,2008-02
2,0,0,0,0,2008-03
3,0,0,0,0,2008-04
4,0,0,0,0,2008-05


# Fill matrice

In [108]:
for index, row in dfAgregateTotal2.iterrows():

    #print 'Years '+str(row['Annee (0)']),'month : '+str( row['mois (0)']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfFinal_2[(dfFinal_2['date']==row['date'])]
    #print 'index : '+str(idx.index.values)
    #print "type d'attaque(0) :" +str(row["type d'attaque(0)"])
    #print 'count : '+str(row['count'])
    dfFinal_2.set_value(idx.index.values, str(row["type d'attaque(0)"]),row['count'])

dfFinal_2.head()

,Attempted,Boarded,Fired upon,Hijacked,date
0,0,0,0,0,2008-01
1,0,0,1,1,2008-02
2,1,1,1,0,2008-03
3,1,0,3,2,2008-04
4,2,0,2,2,2008-05


In [109]:
dfFinal_2['date']=dfFinal_2.apply(lambda row: str(row['date'])+'-01', axis=1)
dfFinal_2['date']=dfFinal_2.apply(lambda row: datetime.datetime.strptime(row['date'], "%Y-%m-%d"), axis=1)
print dfFinal_2.head()

   Attempted  Boarded  Fired upon  Hijacked       date
0          0        0           0         0 2008-01-01
1          0        0           1         1 2008-02-01
2          1        1           1         0 2008-03-01
3          1        0           3         2 2008-04-01
4          2        0           2         2 2008-05-01


In [110]:
dfFinal_2.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)

In [111]:
df["date"].unique()

array(['2010-10', '2010-01', '2010-02', '2010-03', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-11', '2010-04', '2010-12',
       '2008-02', '2008-03', '2008-04', '2008-05', '2008-07', '2008-08',
       '2008-09', '2008-10', '2008-11', '2008-12', '2008-06', '2009-01',
       '2009-02', '2009-03', '2009-04', '2009-05', '2009-07', '2009-12',
       '2009-06', '2009-08', '2009-09', '2009-10', '2009-11', '2011-01',
       '2011-04', '2011-09', '2011-02', '2011-05', '2011-07', '2011-08',
       '2011-10', '2011-03', '2011-11', '2011-06', '2011-12', '2012-02',
       '2012-12', '2012-03', '2012-04', '2012-01', '2012-06', '2012-05',
       '2012-07', '2013-03', '2013-06', '2013-05', '2013-07', '2013-12',
       '2014-01', '2014-04', '2014-02', '2014-03', '2014-08', '2014-12'], dtype=object)